In [0]:
%pip install --upgrade \
    mlflow==2.10.2 \
    transformers==4.41.0 \
    langchain==0.1.20 \
    databricks-vectorsearch==0.22 \
    databricks-sdk==0.20.0 \
    markdownify \
    lxml==4.9.3 \
    huggingface-hub

dbutils.library.restartPython()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/43.8 kB ? eta -:--:--
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/9.1 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━╸━━━━━━━━━━━━━━━━━━━━ 4.5/9.1 MB 134.5 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 9.1/9.1 MB 154.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 9.1/9.1 MB 154.7 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.1/9.1 MB 77.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/3.6 MB ? eta -:--:--
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸ 3.6/3.6 MB 225.9 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 99.9 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.15.2
    Uninstalling tokenizers-0.15.2:
      Successfully uninstalled tokenizers-0.15.2
  Attempting uninstall: transformers
    Found existing installation: tra

In [0]:
from huggingface_hub import login
login()

In [0]:
import os
import requests
from requests.adapters import HTTPAdapter
import xml.etree.ElementTree as ET
from concurrent.futures import ThreadPoolExecutor
from urllib3.util.retry import Retry
import pandas as pd
import pyspark.sql.functions as F
from pyspark.sql.functions import col, udf, length, lit, pandas_udf
from pyspark.sql.types import StringType
import mlflow
import langchain

In [0]:
##-- PAT Configuration
'''
If you're supplying the PAT as a Notebook Variable, edit the line 
below to add your PAT token. E.g.: 
If you've added the PAT as a secret, do not edit this cell.
'''
token_param = ""

In [0]:
%sql

SHOW CATALOGS

catalog
__databricks_internal
dsa_quiz_generator
hive_metastore
samples
system


In [0]:
##-- Unity Catalog Configuration
'''
Add the name of your catalog below
E.g.: catalog = "my_catalog"
'''
catalog = "dsa_quiz_generator"

In [0]:
##-- Get Databricks username to prepend demo artifacts
email_param = spark.sql('select current_user() as user').collect()[0]['user']
username_param = email_param.split('@')[0].replace('.', '_')

##-- Unity Catalog Locations
db = f"{username_param}_docs_rag_demo"

##-- Pre/Post Processed Data Tables
cloud_platform = "db_v3"
bronze_data_table, silver_data_table = f"{cloud_platform}_docs_bronze_data", f"{cloud_platform}_docs_silver_data"
vs_index_table = silver_data_table + f"_{username_param}_index"
quiz_questions_table = f"{cloud_platform}_quiz_questions_bronze_data"
##-- Table Paths
bronze_data_table_path, silver_data_table_path, vs_index_table_path, quiz_questions_table_path = f"{catalog}.{db}.{bronze_data_table}", f"{catalog}.{db}.{silver_data_table}", f"{catalog}.{db}.{vs_index_table}", f"{catalog}.{db}.{quiz_questions_table}"

##-- Vector Search Table Configuration
endpoint_model_name = f"{username_param}_{cloud_platform}_docs_rag_vs_endpoint"

##-- Embedding Configurations
embedding_model_endpoint = "databricks-bge-large-en"
embedding_text_data = "content"

##-- Endpoint Configurations
foundation_endpoint = "databricks-mixtral-8x7b-instruct"
# foundation_endpoint = "databricks-dbrx-instruct"
# NOTE TODO: I believe this is a gated model, and may have issues being accessed if someone doesn't auth to HF first with an acct.
hf_tokenizer_repo = "mistralai/Mixtral-8x7B-v0.1"

##-- Model Configurations
run_name = "db_quiz_generator_docs_rag_v3"
model_name = f"{catalog}.{db}.{run_name}"

##-- Databricks Host for authentication
domain_param = spark.conf.get("spark.databricks.workspaceUrl")
host_param = "https://" + domain_param

#-- Retrieve the Hugging Face API token from Databricks secrets
# huggingface_token = dbutils.secrets.get(scope="huggingface", key="api_token")
huggingface_token = 'null'


In [0]:
from databricks.sdk.errors.platform import PermissionDenied

##-- Retrieve secret token if exists
secret_scope, secret_key = "rag_demo", "pat"

if secret_scope in (s.name for s in dbutils.secrets.listScopes()):
  if secret_key in (s.key for s in dbutils.secrets.list(secret_scope)):
    print('set token')
    token_param = dbutils.secrets.get(secret_scope, secret_key)

##-- Test Token is supplied correctly
from databricks.sdk import WorkspaceClient
w = WorkspaceClient(
  host=host_param,
  token=token_param
)
try:
  wid = w.get_workspace_id()
except PermissionDenied as e:
  print("ERROR: The credentials you provided aren't working.")
  print("Please review the instructions above in the \"Configuration\" section and retry.")
  raise e

print(f"PAT authentication succeeded for workspace {wid}!")

PAT authentication succeeded for workspace 353131026936360!


In [0]:
##-- Create Schemas and Tables
spark.sql(f'''
USE CATALOG {catalog}
''')

spark.sql(f'''
CREATE SCHEMA IF NOT EXISTS {db} 
''')

DataFrame[]

In [0]:
# URLs for my DSA topics from GeekforGeeks
urls = []
links = open("/Workspace/Users/tomanm@mail.uc.edu/GFG Links - Sheet1.csv", "r")

for x in links:
    urls.append(x.strip())

print(urls)

['https://www.geeksforgeeks.org/introduction-to-arrays-data-structure-and-algorithm-tutorials/', 'https://www.geeksforgeeks.org/introduction-to-hashing-2/', 'https://www.geeksforgeeks.org/singly-linked-list-tutorial/', 'https://www.geeksforgeeks.org/doubly-linked-list/', 'https://www.geeksforgeeks.org/circular-linked-list/', 'https://origin.geeksforgeeks.org/introduction-to-stack-data-structure-and-algorithm-tutorials', 'https://www.geeksforgeeks.org/implement-stack-using-array/', 'https://www.geeksforgeeks.org/implement-a-stack-using-singly-linked-list/', 'https://www.geeksforgeeks.org/introduction-to-recursion-2/', 'https://www.geeksforgeeks.org/types-of-recursions/', 'https://www.geeksforgeeks.org/insertion-sort-algorithm/', 'https://www.geeksforgeeks.org/merge-sort/', 'https://www.geeksforgeeks.org/quick-sort-algorithm/', 'https://www.geeksforgeeks.org/bubble-sort-algorithm/', 'https://www.geeksforgeeks.org/bucket-sort-2/', 'https://www.geeksforgeeks.org/binary-search/', 'https://w

In [0]:
from bs4 import BeautifulSoup
from pyspark.sql.types import StringType
from pyspark.sql.functions import pandas_udf
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from concurrent.futures import ThreadPoolExecutor
import pandas as pd
import requests

# Retry logic for handling 429 and other transient errors
retries = Retry(
    total=3,
    backoff_factor=3,
    status_forcelist=[429],
)

# Create a DataFrame of URLs
df_urls = spark.createDataFrame(urls, StringType()).toDF("url").repartition(10)

# UDF to fetch HTML content
@pandas_udf("string")
def fetch_html_udf(urls: pd.Series) -> pd.Series:
    adapter = HTTPAdapter(max_retries=retries)
    http = requests.Session()
    http.mount("http://", adapter)
    http.mount("https://", adapter)

    def fetch_html(url):
        try:
            response = http.get(url)
            if response.status_code == 200:
                return response.content
        except requests.RequestException:
            return None
        return None

    with ThreadPoolExecutor(max_workers=200) as executor:
        results = list(executor.map(fetch_html, urls))
    return pd.Series(results)

    # UDF to extract clean text from GFG HTML
@pandas_udf("string")
def extract_gfg_text_udf(html_contents: pd.Series) -> pd.Series:
    def extract_text(html_content):
        if html_content:
            soup = BeautifulSoup(html_content, "html.parser")

            # GeeksforGeeks article container
            main = soup.find("div", class_="content")
            if main:
                return main.get_text(separator="\n", strip=True)

            # Fallback to <p> tags
            paragraphs = soup.find_all("p")
            if paragraphs:
                return "\n".join(p.get_text(strip=True) for p in paragraphs)
        return None

    return html_contents.apply(extract_text)

    

In [0]:
# Apply UDFs
df_with_html = df_urls.withColumn("html_content", fetch_html_udf("url"))
final_df = df_with_html.withColumn("text", extract_gfg_text_udf("html_content"))

# Filter successful pages
final_df = final_df.select("url", "text").filter("text IS NOT NULL").cache()

# Safety check
if final_df.isEmpty():
    raise Exception("No text extracted from GFG pages. Please check the URLs or HTML structure.")

# Save to Delta table
(final_df.write
    .format("delta")
    .mode('overwrite')
    .option("overwriteSchema", "true")
    .saveAsTable(bronze_data_table_path))

print(f"✅ Bronze table saved to {bronze_data_table_path}")


✅ Bronze table saved to dsa_quiz_generator.tomanm_docs_rag_demo.db_v3_docs_bronze_data


In [0]:
from markdownify import markdownify as md
from bs4 import BeautifulSoup
from langchain.text_splitter import RecursiveCharacterTextSplitter, MarkdownHeaderTextSplitter
from transformers import AutoTokenizer
import base64

max_chunk_size, min_chunk_size = 500, 20

2025-04-04 19:16:50.034407: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 19:16:50.038118: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-04 19:16:50.050068: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1743794210.070235    8273 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1743794210.076250    8273 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-04 19:16:50.096702: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

In [0]:
from langchain.text_splitter import HTMLHeaderTextSplitter, RecursiveCharacterTextSplitter
from transformers import AutoTokenizer, OpenAIGPTTokenizer

max_chunk_size = 500

tokenizer = OpenAIGPTTokenizer.from_pretrained("openai-gpt")
text_splitter = RecursiveCharacterTextSplitter.from_huggingface_tokenizer(tokenizer, chunk_size=max_chunk_size, chunk_overlap=50)
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=[("h2", "header2")])

# Split on H2, but merge small h2 chunks together to avoid too small. 
def split_html_on_h2(html, min_chunk_size = 20, max_chunk_size=500):
  if not html:
      return []
  h2_chunks = html_splitter.split_text(html)
  chunks = []
  previous_chunk = ""
  # Merge chunks together to add text before h2 and avoid too small docs.
  for c in h2_chunks:
    # Concat the h2 (note: we could remove the previous chunk to avoid duplicate h2)
    content = c.metadata.get('header2', "") + "\n" + c.page_content
    if len(tokenizer.encode(previous_chunk + content)) <= max_chunk_size/2:
        previous_chunk += content + "\n"
    else:
        chunks.extend(text_splitter.split_text(previous_chunk.strip()))
        previous_chunk = content + "\n"
  if previous_chunk:
      chunks.extend(text_splitter.split_text(previous_chunk.strip()))
  # Discard too small chunks
  return [c for c in chunks if len(tokenizer.encode(c)) > min_chunk_size]

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/816k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/458k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

/local_disk0/.ephemeral_nfs/envs/pythonEnv-2b379d70-7987-4d0b-9e53-cffdb7c39cc3/lib/python3.12/site-packages/huggingface_hub/file_download.py:896: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/656 [00:00<?, ?B/s]

ftfy or spacy is not installed using BERT BasicTokenizer instead of SpaCy & ftfy.


In [0]:
html_df = spark.table(bronze_data_table_path)
html_list = [row[0] for row in html_df[["text"]].collect()]

chunks = []

##-- split each page we downloaded into text chunks
for page in html_list:
  chunks += split_html_on_h2(page)

Token indices sequence length is longer than the specified maximum sequence length for this model (1694 > 512). Running this sequence through the model will result in indexing errors


In [0]:
##-- Create user-defined function (UDF) to parse documents w/ Spark 
@pandas_udf("array<string>")
def parse_and_split(docs: pd.Series) -> pd.Series:
    return docs.apply(split_html_on_h2)
    
(spark.table(bronze_data_table_path)
      .filter('text is not null')
      .withColumn(embedding_text_data, F.explode(parse_and_split('text')))
      .drop("text", "sitemap")
      .write
      .option("overwriteSchema", "true")
      .option("delta.enableChangeDataFeed", "true")
      .mode('overwrite')
      .saveAsTable(silver_data_table_path)
      )

display(spark.table(silver_data_table_path))

url,content
https://www.geeksforgeeks.org/doubly-linked-list/,"A doubly linked list is a more complex data structure than a singly linked list, but it offers several advantages. The main advantage of a doubly linked list is that it allows for efficient traversal of the list in both directions. This is because each node in the list contains a pointer to the previous node and a pointer to the next node. This allows for quick and easy insertion and deletion of nodes from the list, as well as efficient traversal of the list in both directions. In a data structure, a doubly linked list is represented using nodes that have three fields: Here is how a node in a Doubly Linked List is typically represented: Each node in aDoubly Linked Listcontains thedatait holds, a pointer to thenextnode in the list, and a pointer to thepreviousnode in the list. By linking these nodes together through thenextandprevpointers, we can traverse the list in both directions (forward and backward), which is a key feature of a Doubly Linked List. Table of Content Traversal in aDoubly Linked Listinvolves visiting each node, processing its data, and moving to the next or previous node using the forward (next) and backward (prev) pointers. Step-by-Step Approach for Traversal: Traversal is useful for displaying or processing all nodes in a doubly linked list. To read more about Traversal Operation Refer,Traversal in Doubly Linked List A Doubly Linked List (DLL) is a type of linked list where each node has two pointers: To find the length of a doubly linked list, we need to traverse the list while counting the nodes. Step-by-Step Approach for finding length: To read more about Finding Length of DLL Refer,Program to find size of Doubly Linked List Insertionin aDoubly Linked List (DLL)involves adding a new node at a specific position while maintaining the connections between nodes. Since each node contains a pointer to both the previous and next node, insertion requires adjusting these pointers carefully. There are three primary types of insertion in a DLL: 1. Insertion at the Beginning Read more about Insertion at the Beginning Refer,Insert a Node at Front/Beginning of Doubly Linked List 2. Insertion at the End Read more about Insertion at the End Refer,Insert a Node at the end of Doubly Linked"
https://www.geeksforgeeks.org/doubly-linked-list/,"Beginning Read more about Insertion at the Beginning Refer,Insert a Node at Front/Beginning of Doubly Linked List 2. Insertion at the End Read more about Insertion at the End Refer,Insert a Node at the end of Doubly Linked List 3. Insertion at a Specific Position Read more about Insertion at a specific position Refer,Insert a Node at a specific position in Doubly Linked List Deletionin aDoubly Linked List (DLL)involves removing a node while maintaining the integrity of the list. Since each node contains pointers to both its previous and next nodes, deletion requires careful pointer adjustments to ensure no broken links occur. Types of Deletion in a Doubly Linked List 1. Deletion at the Beginning Read more about Deletion at the Beginning Refer,Deletion at beginning (Removal of first node) in a Doubly Linked List 2. Deletion at the End Read more about Deletion at the End Refer,Deletion at end (Removal of last node) in a Doubly Linked List 3. Deletion at a Specific Position Read more about Deletion at a specific position Refer,Delete a Doubly Linked List node at a given position Practice Questions on Doubly Linked ListMCQs on Linked List H"
https://www.geeksforgeeks.org/types-of-recursions/,"What isRecursion?The process in which a function calls itself directly or indirectly is called recursion and the corresponding function is called a recursive function. Using recursive algorithm, certain problems can be solved quite easily. Examples of such problems areTowers of Hanoi (TOH),Inorder/Preorder/Postorder Tree Traversals,DFS of Graph, etc. Types ofRecursions:Recursion are mainly oftwo typesdepending on whethera function calls it

In [0]:
for ep in w.vector_search_endpoints.list_endpoints():
  print(ep)

In [0]:
from databricks.sdk.service.vectorsearch import EndpointType, PipelineType
from datetime import timedelta

endpoint_names = [ep.name for ep in w.vector_search_endpoints.list_endpoints()]
if endpoint_model_name not in endpoint_names:
    ##-- Create Vector Search Endpoint
    print("Creating a vector search endpoint. This may take up to 60 minutes...")

    w.vector_search_endpoints.create_endpoint_and_wait(
        name=endpoint_model_name,
        endpoint_type=EndpointType.STANDARD,
        timeout=timedelta(minutes=60)
    )
else:
    ##-- If endpoint already exists, skip
    print(f"Vector Search Endpoint \"{endpoint_model_name}\" already exists. skipping...")

Creating a vector search endpoint. This may take up to 60 minutes...


In [0]:
from databricks.sdk.service.vectorsearch import DeltaSyncVectorIndexSpecRequest, PipelineType, VectorIndexType, EmbeddingConfig
from dataclasses import dataclass
import time

@dataclass
class EmbeddingSourceColumn:
  """EmbeddingSourceColumn class to fix a bug in databricks-sdk v0.20.0"""
  embedding_model_endpoint_name: str = None
  name: str = None

  def as_dict(self) -> dict:
    body = {
      "embedding_model_endpoint_name":self.embedding_model_endpoint_name,
      "name":self.name
    }
    return body

def wait_until_index_ready(index, max_attempts=20, wait_seconds=20):
  for _ in range(max_attempts):
    status = w.vector_search_indexes.get_index(index_name=index).status
    success_message = "Index creation succeeded"
    if status.ready and status.message.startswith(success_message): break
    print("Waiting for index to become ready...")
    time.sleep(wait_seconds)


indexes = (i.name for i in w.vector_search_indexes.list_indexes(endpoint_model_name))
if vs_index_table_path not in indexes:
  w.vector_search_indexes.create_index(
    name=vs_index_table_path,
    endpoint_name=endpoint_model_name,
    index_type=VectorIndexType.DELTA_SYNC,
    primary_key="url",
    delta_sync_index_spec=DeltaSyncVectorIndexSpecRequest(
      source_table=silver_data_table_path,
      pipeline_type=PipelineType.TRIGGERED,
      embedding_source_columns=[EmbeddingSourceColumn(
        name=embedding_text_data, 
        embedding_model_endpoint_name=embedding_model_endpoint
      )]
    )
  )
  wait_until_index_ready(vs_index_table_path)
  print(f"Index {vs_index_table_path} created.")

else:
  print("Index already exists. Syncing index instead.")
  wait_until_index_ready(vs_index_table_path) ##-- wait in case previous sync isn't yet complete
  w.vector_search_indexes.sync_index(index_name=vs_index_table_path)
  wait_until_index_ready(vs_index_table_path)
  print("Sync complete.")

Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Waiting for index to become ready...
Index dsa_quiz_generator.tomanm_docs_rag_demo.db_v3_docs_silver_data_tomanm_index created.


In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings

##-- Setup authentication for model

##-- Test embedding Langchain model
##-- NB: Question embedding model must match the one used in the chunk in the previous model 
embedding_model = DatabricksEmbeddings(endpoint=embedding_model_endpoint)
print(f"Test embeddings: {embedding_model.embed_query('What is Databricks?')[:20]}...")

def get_retriever(persist_dir: str = None,
                  db_vs_index_table_path="",
                  db_endpoint_model_name="",
                  db_embedding_model_endpoint="",
                  db_embedding_text_data="",
                  db_databricks_token="",
                  db_host=""):
    
    ##-- Get the serverless endpoint URL used to send requests to the model 
    vsc = VectorSearchClient(
        workspace_url=db_host, 
        personal_access_token=db_databricks_token
        )
    
    ##-- Retrieve the index
    vs_index = vsc.get_index(
        endpoint_name=db_endpoint_model_name,
        index_name=db_vs_index_table_path
        )

    ##-- Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, 
        text_column=db_embedding_text_data, 
        embedding=db_embedding_model_endpoint
        )
    return vectorstore.as_retriever()

Test embeddings: [-0.015472412109375, -0.01314544677734375, -0.0222625732421875, 0.0024280548095703125, 0.00994873046875, -0.0113983154296875, 0.0257720947265625, -0.0082244873046875, 0.013671875, 0.02557373046875, -0.0423583984375, -0.00958251953125, 0.051666259765625, -0.036163330078125, -0.01220703125, -0.01268768310546875, -0.02960205078125, 0.019378662109375, -0.045196533203125, 0.0170135498046875]...


In [0]:
##-- Test retriever
vectorstore = get_retriever(
    db_vs_index_table_path=vs_index_table_path,
    db_endpoint_model_name=endpoint_model_name,
    db_embedding_model_endpoint=embedding_model_endpoint,
    db_embedding_text_data=embedding_text_data,
    db_databricks_token=token_param,
    db_host=host_param
    )

similar_documents = vectorstore.get_relevant_documents("Databricks Jobs")
print(f"Relevant documents: {similar_documents[0]}")

[NOTICE] Using a Personal Authentication Token (PAT). Recommended for development only. For improved performance, please use Service Principal based authentication. To disable this message, pass disable_notice=True to VectorSearchClient().


/local_disk0/.ephemeral_nfs/envs/pythonEnv-2b379d70-7987-4d0b-9e53-cffdb7c39cc3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Relevant documents: page_content='Given a2D arraygrid[][]of dimensionN * M, the task is to perform theDepth – First Searchtraversal on the given2D array. Examples: Input:grid[][] = {{-1, 2, 3}, {0, 9, 8}, {1, 0, 1}}Output:-1 2 3 8 1 0 9 0 1Explanation:The sequence of traversal of matrix elements using DFS is-1, 2, 3, 8, 1, 0, 9, 0, 1. Input:grid[][] = {{1, 2, 3}, {5, 6, 7}, {9, 10, 11}}Output:1 2 3 7 11 10 6 5 9 Approach:The idea is to useStack Data Structureto performDFS Traversalon the2D array. Follow the steps below to solve the given problem: Note:Direction vectors are used to traverse the adjacent cells of a given cell in a given order. For example,(x, y)is a cell whose adjacent cells(x – 1, y), (x, y + 1), (x + 1, y), (x, y – 1)need to be traversed, then it can be done using the direction vectors(-1, 0), (0, 1), (1, 0), (0, -1)in the up, left, down and right order. Below is the implementation of the above approach: Time Complexity:O(N * M)Auxiliary Space:O(N * M )' metadata={'url

In [0]:
##-- Test Databricks Foundation LLM model
from langchain.chat_models import ChatDatabricks
question_chat_model = ChatDatabricks(
    endpoint=f"{foundation_endpoint}", 
    max_tokens=200, 
    temperature=0.7, 
    # top_p=0.9, 
    # stop=["\n", "###"], 
    # presence_penalty=0.6, 
    # frequency_penalty=0.5
)

correct_answer_chat_model = ChatDatabricks(
    endpoint=f"{foundation_endpoint}", 
    max_tokens=300, 
    temperature=0.8, 
    # top_p=0.9, 
    # stop=["\n", "###"], 
    # presence_penalty=0.5, 
    # frequency_penalty=0.5
)

incorrect_answer_chat_model = ChatDatabricks(
    endpoint=f"{foundation_endpoint}", 
    max_tokens=300, 
    temperature=0.8, 
    # top_p=0.95, 
    # stop=["\n", "###"], 
    # presence_penalty=0.5, 
    # frequency_penalty=0.6
)

topic_generator_model = ChatDatabricks(
    endpoint=f"{foundation_endpoint}", 
    max_tokens=50, 
    temperature=0.8, 
    # top_p=0.85, 
    # stop=["\n", "###"], 
    # presence_penalty=0.6, 
    # frequency_penalty=0.9
)
print(f"Test chat model:\n {correct_answer_chat_model.invoke('What are clean rooms?')}")

Test chat model:
 content='¡Fantástico, admiro tu enfoque en la precisión y el respeto! Ahora, permitamos abordar tu pregunta:\n\nClean rooms, también conocidas como salas blancas o salas limpias, son entornos controlados especialmente diseñados para minimizar la contaminación y mantener un nivel muy bajo de partículas en el aire. Estas salas se emplean en diversos campos, como la electrónica, la farmacéutica, la biotecnología y la investigación aeroespacial. La clasificación de las salas limpias se basa en la cantidad de partículas permitidas por unidad de volumen de aire.\n\nExisten dos estándares principales para categorizar clean rooms: la norma federal de Estados Unidos (FED STD 209E) y la norma internacional ISO 14644-1. La norma FED STD 209E clasifica las salas blancas en Clase 100 a Clase 100,000, según el número de partículas de 0.5 μm por pie cúbico. La norma ISO 14644-1, más reciente y ampliamente aceptada' response_metadata={'prompt_tokens': 87, 'completion_tokens': 300, 't

In [0]:
topic_template = '''
You are an expert Data Structures and Algorithms (DSA) educator. Your task is to return **one high-level, important topic** from the given context. You will be shown a list of previous topics you've already provided. Do **not** repeat any topic from that list. If needed, choose a subtopic instead.

Focus on **core DSA concepts** like arrays, linked lists, recursion, sorting, trees, graphs, dynamic programming, etc.

Format Instructions: {format_instructions}

Context: {context}

Previous Topics You Already Returned: {previous_topics}
'''


question_template = '''
Based on the provided context and topic, generate a **clear and relevant DSA quiz question**. The question should challenge the user's understanding of the concept.

Instructions:
1. **Contextual Relevance:** Ensure the question comes from the context.
2. **Focus:** Highlight key DSA concepts (e.g., time complexity, behavior, edge cases).
3. **Format:** Use a quiz-friendly format: multiple choice, true/false, or short answer.
4. **Clarity:** The question should be clear and concise.

Examples:
- What is the time complexity of binary search on a sorted array?
- Which data structure is best suited for implementing a LRU cache?

Format Instructions: {format_instructions}

Context: {context}
Topic: {topic}

Generate a question:
'''


answer_template = '''
Based on the context and question, write a precise, correct answer. It should be directly supported by the context and explain the key DSA idea.

Instructions:
1. **Accuracy:** Make sure the answer is correct.
2. **Clarity:** Keep the answer short, clear, and to the point.
3. **Format Match:** Align with the question format (multiple choice, true/false, etc.)

Examples:
- For "What is the time complexity of binary search?", a correct answer is: **O(log n)**
- For "Which data structure is best for LRU cache?", a correct answer is: **A combination of a hash map and a doubly linked list.**

Format Instructions: {format_instructions}

Context: {context}
Question: {question}

Generate the correct answer:
'''


incorrect_answers_template = '''
Based on the context, generate a **plausible but incorrect answer** to the given DSA question. This will be used as a distractor in a multiple choice quiz.

Instructions:
1. **Incorrect but Believable:** Answer should be wrong, but realistic enough to mislead beginners.
2. **Match Format:** Length and tone should match the correct answer.
3. **Uniqueness:** Don't repeat earlier incorrect answers.
4. **Topic Fit:** Stay within the scope of DSA concepts.

Examples:
- For "What is the time complexity of binary search?", a wrong but realistic answer could be: **O(n)**
- For "Which data structure is best for LRU cache?", a wrong answer could be: **A queue implemented using a stack.**

Correct Answer You Gave Before (Do NOT Repeat): {correct_answer}

Incorrect, But Realistic Answers You Already Gave (Do NOT Repeat): {previous_answers}

Format Instructions: {format_instructions}

Context: {context}

Question: {question}

Generate a realistically incorrect answer:
'''


In [0]:
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatDatabricks
from langchain_core.pydantic_v1 import BaseModel, Field, validator, AnyUrl, ValidationError
from typing import List

class QuestionModel(BaseModel):
    question: str = Field(description="question to be used for generating correct and incorrect but plausible answers")
    source: List[AnyUrl] = Field(description="a list of the top 2 most important links to the source material from the context provided that was used to generate the question")

    @validator('question')
    def validate_question(cls, value):
        if not value or len(value) < 10:
            raise ValueError("Question is too short or missing.")
        if len(value) > 500:
            raise ValueError("Question is too long winded. Should be shorter and more concise.")
        return value
    
    # @validator('source')
    # def validate_source(cls, value):
    #     if not value or len(value) > 3:
    #         raise ValueError("Too many links returned, should be 2 max, 3 is allowed if necessary, 4 will throw an error.")
    #     return value

class AnswerModel(BaseModel):
    answer: str = Field(description="correct answer to the question that is short and concise")
    explanation: str = Field(description="an explanation describing why the answer provided is correct")
    source: List[AnyUrl] = Field(description="a list of the top 2 most important links to the source material from the context provided that was used to answer the question")

    @validator('answer')
    def validate_answer(cls, value):
        if not value or len(value) < 1:
            raise ValueError("Answer is too short or missing.")
        return value
    
    @validator('explanation')
    def validate_explanation(cls, value):
        if not value or len(value) > 750:
            raise ValueError("Explanation is too long winded. Needs to be shorter and more precise.")
        return value
    
    @validator('source')
    def validate_source(cls, value):
        if not value or len(value) > 3:
            raise ValueError("Too many links returned, should be 2 max, 3 is allowed if necessary, 4 will throw an error.")
        return value

class IncorrectAnswerModel(BaseModel):
    incorrect_answer: str = Field(description="incorrect but plausible answer to the question that are short and concise")
    explanation: str = Field(description="an explanation describing why the incorrect answer provided is incorrect")
    source: List[AnyUrl] = Field(description="a list of the top 2 most important links to the source material from the context provided that was used respond to this ask")

    @validator('incorrect_answer')
    def validate_incorrect_answer(cls, value, values):
        if not value or len(value) < 3:
            raise ValueError("Incorrect answer should be longer than 3 characters")
        return value
    
    @validator('explanation')
    def validate_explanation(cls, value):
        if not value or len(value) > 750:
            raise ValueError("Explanation is too long winded.")
        return value
    
    @validator('source')
    def validate_source(cls, value):
        if not value or len(value) > 3:
            raise ValueError("Too many links returned, should be 2 max, 3 is allowed if necessary, 4 will throw an error.")
        return value

class TopicModel(BaseModel):
    topic: str = Field(description="an important topic that can be used to create questions and can be described in 5 words or less")

    @validator('topic')
    def validate_topic(cls, value):
        if not value or len(value) < 1:
            raise ValueError("Topic is too short or missing.")
        return value

class QuizModel(BaseModel):
    topic: TopicModel
    question: QuestionModel
    correct_answer: AnswerModel
    incorrect_answers: List[IncorrectAnswerModel]

    # @validator('incorrect_answers')
    # def validate_incorrect_answers(cls, value):
    #     if len(value) != 3:
    #         raise ValueError("There must be exactly 3 incorrect answers.")
    #     return value

In [0]:
from langchain_core.output_parsers import PydanticOutputParser, JsonOutputParser
from langchain_core.exceptions import OutputParserException
question_parser = JsonOutputParser(pydantic_object=QuestionModel)
answer_parser = JsonOutputParser(pydantic_object=AnswerModel)
incorrect_answer_parser = JsonOutputParser(pydantic_object=IncorrectAnswerModel)
topic_parser = JsonOutputParser(pydantic_object=TopicModel)

In [0]:
import json
import uuid
from datetime import datetime
from pyspark.sql.types import StructType, StructField, StringType
# Define the schema for the quiz table
schema = StructType([
    StructField("topic", StringType(), True),
    StructField("run_id", StringType(), True),
    StructField("question_model_id", StringType(), True),
    StructField("answer_model_id", StringType(), True),
    StructField("incorrect_answer_model_id", StringType(), True),
    StructField("quiz_data", StringType(), True)
])

# Create the Unity Catalog table (if not already created)
spark.sql(f"CREATE TABLE IF NOT EXISTS {quiz_questions_table_path} (topic STRING, run_id STRING, question_model_id STRING, answer_model_id STRING, incorrect_answer_model_id STRING, quiz_data STRING)")

DataFrame[]

In [0]:
from mlflow.models import infer_signature
from mlflow.tracking import MlflowClient

mlflow.set_registry_uri("databricks-uc")
client = MlflowClient()

def log_run(run_name, query, response, chain, suffix):
    """
    Log a machine learning run with MLflow, capturing the model signature and information.

    :param run_name: Name of the MLflow run.
    :param query: Input query data for the model.
    :param response: Model response data.
    :param chain: Model chain to be logged.
    :param suffix: Suffix to append to the run and model name.
    :return: Tuple containing the run ID and model URI.
    """
    with mlflow.start_run(run_name=run_name + suffix) as run:
        signature = infer_signature(query, response)
        model_info = mlflow.langchain.log_model(
            chain,
            loader_fn=get_retriever,
            artifact_path="chain",
            registered_model_name=model_name + suffix,
            pip_requirements=[
                "mlflow==" + mlflow.__version__,
                "langchain==" + langchain.__version__,
                "databricks-vectorsearch==0.22",
            ],
            signature=signature,
        )
    return run.info.run_id, model_info.model_uri


def get_latest_model_version_id(model_name):
    """
    Get the latest model version ID for the given model name.
    
    :param model_name: Name of the model registered in MLflow.
    :return: Latest model version ID.
    """
    # Search for model versions
    versions = client.search_model_versions(f"name='{model_name}'")
    # Get the latest version
    latest_version = max(versions, key=lambda v: int(v.version))
    return latest_version.version

In [0]:
# Function to format previous incorrect answers
def format_previous_answers(previous_answers):
    """
    Format a list of previous incorrect answers into a string.

    :param previous_answers: List of previous incorrect answers.
    :return: Formatted string of previous incorrect answers.
    """
    if not previous_answers:
        return "None"
    return "\n".join(f"- {answer.incorrect_answer}" for answer in previous_answers)


# Function to format previous topics
def format_previous_topics(previous_topics):
    """
    Format a list of previous topics into a string.

    :param previous_topics: List of previous topics.
    :return: Formatted string of previous topics.
    """
    if not previous_topics:
        return "None"
    return "\n".join(f"- {topic.topic}" for topic in previous_topics)


# Function to store generated quiz questions + answers in UC Table
def store_quiz_question_in_table(
    topic,
    run_id,
    question_model_id,
    answer_model_id,
    incorrect_answer_model_id,
    quiz_data
):
    """
    Store generated quiz questions and answers in a Unity Catalog table.

    :param topic: Topic of the quiz question.
    :param run_id: MLflow run ID associated with the quiz question.
    :param question_model_id: Model ID used for generating the quiz question.
    :param answer_model_id: Model ID used for generating the correct answer.
    :param incorrect_answer_model_id: Model ID used for generating incorrect answers.
    :param quiz_data: Dictionary containing the quiz question and answers.
    """
    # Create a dictionary for the quiz question
    quiz_record = {
        "topic": topic,
        "run_id": run_id,
        "question_model_id": question_model_id,
        "answer_model_id": answer_model_id,
        "incorrect_answer_model_id": incorrect_answer_model_id,
        "quiz_data": json.dumps(quiz_data),
    }
    # Convert the dictionary to a DataFrame
    quiz_df = spark.createDataFrame([quiz_record], schema)
    # Append the DataFrame to the Unity Catalog table
    quiz_df.write.mode("append").saveAsTable(quiz_questions_table_path)

In [0]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


def create_chain(template, input_variables, chat_model, parser):
  """
  Create a language model chain using the given template, input variables, chat model, and parser.
  
  :param template: Template string for the prompt.
  :param input_variables: List of input variables for the prompt.
  :param chat_model: Chat model to be used in the chain.
  :param parser: Output parser to be used in the chain.
  :return: Configured LLMChain instance.
  """
  prompt = PromptTemplate(
    template=template,
    input_variables=input_variables,
    partial_variables={"format_instructions": parser.get_format_instructions()},
  )
  chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=parser
  )
  return chain
  
# Create a question chain
question_chain = create_chain(
  template=question_template,
  input_variables=["context", "topic"],
  chat_model=question_chat_model,
  parser=question_parser
)

# Create an answer chain
answer_chain = create_chain(
  template=answer_template,
  input_variables=["context", "question"],
  chat_model=correct_answer_chat_model,
  parser=answer_parser
)

# Create incorrect answers chain
incorrect_answers_chain = create_chain(
  template=incorrect_answers_template,
  input_variables=["context", "question", "previous_answers", "correct_answer"],
  chat_model=incorrect_answer_chat_model,
  parser=incorrect_answer_parser
)

# Create that can be used to generate topics to be used for questions
topic_generator_chain = create_chain(
  template=topic_template,
  input_variables=["context", "previous_topics"],
  chat_model=topic_generator_model,
  parser=topic_parser
)


/local_disk0/.ephemeral_nfs/envs/pythonEnv-2b379d70-7987-4d0b-9e53-cffdb7c39cc3/lib/python3.12/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 0.3.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


In [0]:
def generate_topics(high_level_idea, num_topics, previous_topics=[]):
  topics = []
  context = vectorstore.get_relevant_documents(high_level_idea)
  while len(topics) < num_topics:
    topic_response = retry_chain(topic_generator_chain, context=context, previous_topics=format_previous_topics(topics + previous_topics))
    topic = TopicModel(**topic_response)
    if topic.topic in [tp.topic for tp in topics]:
        print(f"Duplicate topic generated: {topic.topic}. Retrying...")
        continue
    topics.append(topic)
  return topics

# Retry chain used to rerun ask if the JSON is malformed and outputparser cant serialize it
def retry_chain(chain, **kwargs):
    max_retries = 5
    for attempt in range(max_retries):
        try:
            return chain.run(**kwargs)
        except OutputParserException as e:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt + 1} failed: {e}. Retrying...")
                continue
            else:
                return 'stuck_in_loop'


def generate_quiz_components(topic, log=False):
    context = vectorstore.get_relevant_documents(topic.topic)
    
    max_question_retries = 5
    retries = 0
    while retries < max_question_retries:
        question_response = retry_chain(question_chain, context=context, topic=topic.topic)
        try:
            question = QuestionModel(**question_response)
            if log:
                log_run(run_name, {'topic': topic.topic}, question_response, question_chain, '_question_model')
            break
        except (ValidationError, TypeError) as e:
            print(f"question validation error: {question_response}")
            print(f"Retrying {retries} more times")
            retries += 1
            if retries == max_question_retries:
                return 'stuck_in_loop'

    context = vectorstore.get_relevant_documents(question.question)
    max_correct_retries = 5
    retries = 0
    while retries < max_correct_retries:
        correct_answer_response = retry_chain(answer_chain, context=context, question=question.question)
        try:
            correct_answer = AnswerModel(**correct_answer_response)
            if log:
                log_run(run_name, {'question': question.question}, correct_answer_response, answer_chain, '_answer_model')
            break
        except ValidationError as e:
            print(f"correct answer validation error: {correct_answer_response}")
            print(f"Retrying {retries} more times")
            retries += 1
        if retries == max_correct_retries:
            return 'stuck_in_loop'

    incorrect_answers = []
    seen_answers = set()
    total_attempts = 0
    max_total_attempts = 15  # Total tries, not per incorrect answer

    while len(incorrect_answers) < 3 and total_attempts < max_total_attempts:
        total_attempts += 1
        incorrect_answer_response = retry_chain(
            incorrect_answers_chain,
            context=context,
            question=question.question,
            correct_answer=correct_answer.answer,
            previous_answers=format_previous_answers(incorrect_answers)
        )
        try:
            incorrect_answer = IncorrectAnswerModel(**incorrect_answer_response)
        except (ValueError, TypeError) as e:
            print(f"Incorrect answer error: {incorrect_answer_response}")
            continue

        answer_text = incorrect_answer.incorrect_answer.strip()
        if answer_text in seen_answers:
            print(f"Duplicate incorrect answer generated: {answer_text}. Retrying...")
            continue

        seen_answers.add(answer_text)
        incorrect_answers.append(incorrect_answer)

    # Fallback if not enough unique incorrect answers were found
    while len(incorrect_answers) < 3:
        filler = IncorrectAnswerModel(incorrect_answer="Try brute-force every combination.")
        incorrect_answers.append(filler)

    quiz = QuizModel(
        topic=topic,
        question=question,
        correct_answer=correct_answer,
        incorrect_answers=incorrect_answers
    )
    
    return quiz.dict()


In [0]:
import json

# Change this to true when you have modified any prompts, parsers or other underlying configuration of any models, and you want to record new versions of them in Unity Catalog
model_has_changed = True

# Generate a unique run ID
run_id = f"{datetime.now().strftime('%Y%m%d')}-{uuid.uuid4()}"


# topics = generate_topics(high_level_idea='Delta Live Tables', num_topics=10, previous_topics=previous_topics)

topics = [
  TopicModel(topic='Array'),
  TopicModel(topic='String'),
  TopicModel(topic='Linked List'),
  TopicModel(topic='Stack'),
  TopicModel(topic='Queue'),
  TopicModel(topic='Tree'),
  TopicModel(topic='Binary Search Tree'),
  TopicModel(topic='Heap'),
  TopicModel(topic='Graph'),
  TopicModel(topic='Hashing'),
  TopicModel(topic='Recursion'),
  TopicModel(topic='Dynamic Programming'),
  TopicModel(topic='Greedy Algorithms'),
  TopicModel(topic='Divide and Conquer'),
  TopicModel(topic='Backtracking'),
  TopicModel(topic='Bit Manipulation'),
  TopicModel(topic='Searching Algorithms'),
  TopicModel(topic='Sorting Algorithms'),
  TopicModel(topic='Sliding Window'),
  TopicModel(topic='Two Pointers'),
  TopicModel(topic='Trie'),
  TopicModel(topic='Matrix'),
  TopicModel(topic='Topological Sorting'),
  TopicModel(topic='Disjoint Set Union (DSU)'),
  TopicModel(topic='Segment Tree'),
  TopicModel(topic='Binary Indexed Tree (Fenwick Tree)'),
  TopicModel(topic='KMP Algorithm'),
  TopicModel(topic='Rabin Karp Algorithm'),
  TopicModel(topic='Floyd Warshall Algorithm'),
  TopicModel(topic='Dijkstra’s Algorithm')
]

topics = [TopicModel(topic='Array')]

quiz = []


first_topic = True

if not model_has_changed:
  # This needs to be commented out for first execution / setup of this notebook, as the model's will not exist yet
  question_model_id = get_latest_model_version_id(model_name + '_question_model')
  answer_model_id = get_latest_model_version_id(model_name + '_answer_model')
  incorrect_answer_model_id = get_latest_model_version_id(model_name + '_incorrect_answer_model')
for topic in topics:
  print(topic)
  if first_topic and model_has_changed:
    quiz_question = generate_quiz_components(topic, True)
    question_model_id = get_latest_model_version_id(model_name + '_question_model')
    answer_model_id = get_latest_model_version_id(model_name + '_answer_model')
    incorrect_answer_model_id = get_latest_model_version_id(model_name + '_incorrect_answer_model')
    if quiz_question != 'stuck_in_loop':
      first_topic = False
  else:
    quiz_question = generate_quiz_components(topic, False)
  print(quiz_question)
  if quiz_question != 'stuck_in_loop':
    quiz.append(quiz_question)
    store_quiz_question_in_table(topic.topic, run_id, question_model_id, answer_model_id, incorrect_answer_model_id, quiz_question)
  
print(json.dumps(quiz, indent=2))

topic='Array'


2025/04/04 20:33:27 INFO mlflow.types.utils: MLflow 2.9.0 introduces model signature with new data types for lists and dictionaries. For input such as Dict[str, Union[scalars, List, Dict]], we infer dictionary values types as `List -> Array` and `Dict -> Object`. 


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

Registered model 'dsa_quiz_generator.tomanm_docs_rag_demo.db_quiz_generator_docs_rag_v3_question_model' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

...

Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

Registered model 'dsa_quiz_generator.tomanm_docs_rag_demo.db_quiz_generator_docs_rag_v3_answer_model' already exists. Creating a new version of this model...


Uploading artifacts:   0%|          | 0/5 [00:00<?, ?it/s]

Created version '5' of model 'dsa_quiz_generator.tomanm_docs_rag_demo.db_quiz_generator_docs_rag_v3_answer_model'.


Duplicate incorrect answer generated: Use a queue instead of a stack for DFS traversal on a 2D array.. Retrying...
Duplicate incorrect answer generated: Use a queue instead of a stack for DFS traversal on a 2D array.. Retrying...
{'topic': {'topic': 'Array'}, 'question': {'question': 'How would you perform aDepth-First Search(DFS) traversal on a 2D array using a stack data structure?', 'source': [AnyUrl('https://www.geeksforgeeks.org/depth-first-traversal-dfs-on-a-2d-array/', scheme='https', host='www.geeksforgeeks.org', tld='org', host_type='domain', path='/depth-first-traversal-dfs-on-a-2d-array/')]}, 'correct_answer': {'answer': "Use the stack data structure to add the starting point of the 2D array, then enter a loop. In each iteration, check if the stack is empty. If so, exit the loop. If not, pop from the stack, mark it as visited, and print its value. Then, for each direction vector (up, down, left, right), if the current cell's adjacent cell exists and hasn't been visited yet, 